In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
from PIL import Image, ImageChops, ImageDraw
import random as rand
import math
from IPython import display

## 0. Utility

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def randfloat(a, b):
    return rand.random() * (b - a) + a

def my_print(s):
    sys.stdout.write(s)
    sys.stdout.flush()

## 1. Generate background

In [4]:
raw_dir = "./backgrounds_raw/"
back_dir = "./backgrounds/"

back_size = (64, 64)

create_dir(back_dir)

In [ ]:
def handle_raw(image, cur_idx):
    image = image.convert("RGB")
    image = image.resize((back_size[0] * 4, back_size[1] * 4))
    size = image.size
    cx = size[0] / back_size[0]
    cy = size[1] / back_size[1]
#     if cur_idx + cx * cy <= skip_N:
#         cur_idx += cx * cy
#         return cur_idx

    for i in xrange(cx):
        for j in xrange(cy):
            image_tmp = image.transform(back_size, Image.EXTENT, (back_size[0] * i, back_size[1] * j, 
                                                          back_size[0] * (i + 1), back_size[1] * (j + 1)))
            image_tmp.save(os.path.join(back_dir, str(cur_idx) + ".png"), "PNG")
            cur_idx += 1
            
            if cur_idx % 1000 == 0:
                my_print("\rcur_idx = %i" % cur_idx)
    
    return cur_idx

In [ ]:
N = 0
for block_name in os.listdir(raw_dir):
    block_dir = os.path.join(raw_dir, block_name)
    for image_name in os.listdir(block_dir):
        image_path = os.path.join(block_dir, image_name)
        image = Image.open(image_path)
        N = handle_raw(image, N)
        

In [ ]:
print N

total: 116102

## 2. Generate input

In [5]:
arrows_dir = "./arrows/"

input_dir = "../input/"
create_dir(input_dir)

input_size = back_size

arrow_size = (16, 16)

min_speed = 5
max_speed = 30
min_scale = .5
max_scale = 1.5

In [6]:
arrows = []
for arrow_name in os.listdir(arrows_dir):
    arrow_path = os.path.join(arrows_dir, arrow_name)
    image = Image.open(arrow_path)
    image = image.split()[0].convert("1")
    image = ImageChops.invert(image)
    image = image.resize(arrow_size, Image.ANTIALIAS)
    arrows.append(image)
    
arrow_cnt = len(arrows)
print arrow_cnt

4


In [7]:
def handle_true(back_image):
    top_image = back_image.copy()
    bot_image = back_image.copy()
    top_draw = ImageDraw.Draw(top_image)
    bot_draw = ImageDraw.Draw(bot_image)
    cnts = [1, 1, 1, 2, 2, 3]
    arrow_cnt = cnts[rand.randint(0, len(cnts) - 1)]
    direction = rand.randint(0, 1) * 2 - 1
    for i in xrange(arrow_cnt):
        cur_arrow = arrows[rand.randint(0, arrow_cnt - 1)].copy()
        cur_arrow = cur_arrow.resize((int(cur_arrow.size[0] * randfloat(min_scale, max_scale)),
                                      int(cur_arrow.size[1] * randfloat(min_scale, max_scale))))
        arrow_color = (rand.randint(0, 255), rand.randint(0, 255), rand.randint(0, 255))
        angle = rand.random() * 360
        cur_arrow = cur_arrow.rotate(angle, expand=True)
        x0 = rand.randint(0, back_image.size[0] - cur_arrow.size[0] - 1)
        y0 = rand.randint(0, back_image.size[1] - cur_arrow.size[1] - 1)
        speed = rand.randint(min_speed, max_speed)
        x1 = int(x0 + direction * speed * math.cos(angle * math.pi / 180))
        y1 = int(y0 - direction * speed * math.sin(angle * math.pi / 180))
        
#         print arrow_color, x0, y0
        top_draw.bitmap((x0, y0), cur_arrow, arrow_color)
        bot_draw.bitmap((x1, y1), cur_arrow, arrow_color)
        
    full_image = Image.new("RGB", (2 * back_image.size[0], back_image.size[1]))
    full_image.paste(top_image, (0, 0))
    full_image.paste(bot_image, (back_image.size[0], 0))

    return full_image, direction
    

In [ ]:
"""FOR REAL BACKGROUNDS"""
classes = open("../labels.csv", "w+")
for i, back_name in enumerate(os.listdir(back_dir)):
    if i % 1000 == 0:
        my_print("\r i = %i" % i)
    back_path = os.path.join(back_dir, back_name)
    back_image = Image.open(back_path).convert("RGB")
    X, y = handle_true(back_image)
    X.save(os.path.join(input_dir, str(i) + ".png"), "PNG")
    classes.write(str(i) + "," + str(y) + "\n")
    
classes.close()
    

In [8]:
"""FOR BLACK BACKGROUNDS"""
classes = open("../labels.csv", "w+")
for i in xrange(10000):
    if i % 1000 == 0:
        my_print("\r i = %i" % i)
    back_image = Image.new("RGB", back_size)
    X, y = handle_true(back_image)
    X.save(os.path.join(input_dir, str(i) + ".png"), "PNG")
    classes.write(str(i) + "," + str(y) + "\n")
    
classes.close()
    

 i = 9000

In [15]:
k = rand.randint(0, 10000)
img = Image.open("../input/" + str(k) + ".png") 
img.show()